In [ ]:
# 호환성 오류가 발생할 경우, 다음의 pip 구문을 순차적으로 실행 

# !pip install pydantic==1.10.9
# !pip install --force-reinstall chromadb

In [ ]:
# install 진행 후 아래 테스트 코드 실행하여 
# 1. 프린트 구문이 정상 출력되는지 확인 
# 2. 경로 내에 'test_chroma' 경로가 활성화 되는지 확인 

# from langchain.vectorstores import Chroma

# 간단한 테스트용 Chroma 벡터스토어 생성
# chroma_db = Chroma(
#     collection_name="test_collection",
#     persist_directory="./test_chroma"
# )
# print("Chroma 벡터스토어 생성 성공!")

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# Chroma

1. 개요
    - LLM과 자연어 처리 어플리케이션 위한 벡터 데이터베이스 
    - FAISS는 고성능 벡터 검색과 대규모 데이터 처리에 적합
    - Chroma는 NLP에 최적화된 벡터 데이터베이스로 메타데이터 통합 관리에 강점

2. FAISS vs Chroma

 | 특징 | FAISS | Chroma |
 |-----|-------|--------|
 | 데이터 저장 | 메모리 기반, 디스크 저장은 제한적 | 디스크 기반 영구 저장 지원 |
 | 메타데이터 관리 | 별도 관리 필요 | 벡터와 메타데이터 통합 관리 |
 | 검색 알고리즘 | 다양한 ANN 인덱스 제공 | 유사도 검색 + 메타데이터 필터링 |
 | 성능 | GPU 통한 대규모 데이터 검색 | CPU 기반으로 상대적으로 느림 |


[Reference] https://docs.trychroma.com/docs/overview/getting-started

[Reference] https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/

In [3]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# 파일 경로 저장 
file_path1 = './data/nlp-keywords.txt'
file_path2 = './data/finance-keywords.txt'

In [5]:
# 분할기 정의 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 파일 로딩 
loader1 = TextLoader(file_path1)
loader2 = TextLoader(file_path2)

# 텍스트 분할 
text_split1 = loader1.load_and_split(text_splitter)
text_split2 = loader2.load_and_split(text_splitter)

In [6]:
# 분할된 파일 개수 확인 
len(text_split1), len(text_split2)

(7, 4)

## DB 저장

In [7]:
# 벡터DB 정의 
# collection : 폴더 
vector_store = Chroma.from_documents(
    documents=text_split1,
    embedding=OpenAIEmbeddings(),
    collection_name='chroma_db',
)

In [8]:
# DB 경로 정의 
chroma_path = './chroma_db'

# 문서 저장 
# persist_directory 지정하면 디스크에 파일 형태로 저장됨 
vector_store_persist = Chroma.from_documents(
    documents=text_split1,
    embedding=OpenAIEmbeddings(),
    persist_directory=chroma_path,
    collection_name='chroma_db',

)

In [9]:
# 저장된 데이터 확인 
vector_store_persist.get()

{'ids': ['cac1939b-32e3-412d-ab2d-8abc3170d693',
  '095cf91d-2166-4c00-8bbe-093ead5bbed0',
  '2351d465-d439-4ed8-a182-93b6fef397b3',
  '8b76c30e-ca99-4129-92bd-659fc25cf55d',
  'dcb3e163-bd71-4b59-a98a-7dd48e4ec17d',
  'c45e7881-852d-4a5a-ae82-4c3ac66196f7',
  'aa56b365-f74f-407f-87d0-8364a295e84e',
  'text2_doc_0',
  'text2_doc_1',
  'c7d3e6ac-d63b-4691-a0a3-57d5e0d82def',
  'fa45f71e-0e79-4147-a12b-135e3a0c67f5',
  '92a67cc7-deb6-4224-97a4-d4c3add54f2f',
  '918328ce-4815-4c5c-9bbd-6aa4e273532d',
  '6f3b6142-2172-4d22-8812-215c212730ba',
  '2f07cd6e-23fb-4dd4-860b-8a66bb9f0592',
  '7eb87de1-dfbc-4efe-9389-ef3895c6d458'],
 'embeddings': None,
 'documents': ['Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 

## 유사도 검색

`similarity_search` 메서드는 주어진 쿼리와 가장 유사한 문서들을 검색하는 기능 제공

1. 매개변수
    - `query` (str): 검색할 쿼리 텍스트
    - `k` (int, 선택적): 반환할 결과의 수. 기본값은 4
    - `filter` (Dict[str, str], 선택적): 메타데이터로 필터링. 기본값은 None

2. 반환값
    - `List[Document]`: 쿼리와 가장 유사한 문서 리스트

In [10]:
# 유사도 검색
vector_store_persist.similarity_search("TF IDF 에 대하여 알려줘")

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을

In [11]:
# k 값 지정
vector_store_persist.similarity_search("TF IDF 에 대하여 알려줘", k=1)

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을

In [12]:
# filter 사용
vector_store_persist.similarity_search(
    "TF IDF 에 대하여 알려줘", 
    filter={"source": file_path1}, # nlp-keywords.txt
    k=2,
)

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을

In [13]:
# filter 사용
vector_store_persist.similarity_search(
    "TF IDF 에 대하여 알려줘", 
    filter={"source": file_path2}, # finance-keywords.txt
    k=2,
)

[Document(metadata={'source': './data/finance-keywords.txt'}, page_content='Price-to-Earnings Ratio (P/E Ratio)\n\n정의: 주가수익비율(P/E)은 주가를 주당순이익으로 나눈 값으로, 기업의 가치를 평가하는 데 사용되는 지표입니다.\n예시: 아마존의 P/E 비율이 높은 것은 투자자들이 회사의 미래 성장 가능성을 높게 평가하고 있다는 것을 의미합니다.\n연관키워드: 주식 가치평가, 투자 분석, 성장주\n\nQuarterly Earnings Report\n\n정의: 분기별 실적 보고서는 기업이 3개월마다 발표하는 재무 성과와 사업 현황에 대한 보고서입니다.\n예시: 애플의 분기별 실적 발표는 전체 기술 섹터와 S&P 500 지수에 큰 영향을 미칩니다.\n연관키워드: 기업 실적, 투자자 관계, 재무 분석\n\nIndex Fund\n\n정의: 인덱스 펀드는 S&P 500과 같은 특정 지수의 구성과 성과를 그대로 추종하는 투자 상품입니다.\n예시: 바운가드 S&P 500 ETF는 S&P 500 지수를 추종하는 대표적인 인덱스 펀드입니다.\n연관키워드: 패시브 투자, ETF, 포트폴리오 관리\n\nMarket Weight\n\n정의: 시장 가중치는 특정 기업이나 섹터가 전체 지수에서 차지하는 비중을 나타냅니다.\n예시: 기술 섹터는 S&P 500 지수에서 가장 큰 시장 가중치를 차지하고 있습니다.\n연관키워드: 포트폴리오 구성, 섹터 분석, 자산 배분\n\nGrowth Stock\n\n정의: 성장주는 평균 이상의 높은 성장률을 보이는 기업의 주식을 의미합니다.\n예시: 페이스북(메타)과 같은 기술 기업들은 S&P 500에 포함된 대표적인 성장주로 꼽힙니다.\n연관키워드: 고성장 기업, 기술주, 투자 전략\n\nValue Stock\n\n정의: 가치주는 현재 시장 가치가 내재 가치보다 낮다고 평가되는 기업의 주식을 말합니다.\n예시: 워렌 버핏이 투자한 코카콜라는 S&P 500에 포함된

## 문서 추가 

`add_documents` 메서드는 벡터 저장소에 문서를 추가하거나 업데이트하는 기능 제공

In [14]:
# 추가된 문서임을 구문하기 위해 ID를 구분하고자 함 
ids = [f"text2_doc_{i}" for i in range(len(text_split2))]

# DB에 문서 추가 
vector_store_persist.add_documents(text_split2, ids=ids)

['text2_doc_0', 'text2_doc_1', 'text2_doc_2', 'text2_doc_3']

In [15]:
# 추가된 문서 확인 
vector_store_persist.get('text2_doc_0')

{'ids': ['text2_doc_0'],
 'embeddings': None,
 'documents': ['S&P 500\n\n정의: S&P 500은 미국 주식 시장에 상장된 500개의 대형 기업의 주가를 종합한 지수입니다. 이는 미국 경제와 주식 시장의 전반적인 상황을 나타내는 주요 지표로 사용됩니다.\n예시: 애플, 마이크로소프트, 아마존과 같은 대형 기술 기업들이 S&P 500에 포함되어 있습니다.\n연관키워드: 주식 시장, 지수, 대형주\n\nMarket Capitalization\n\n정의: 시가총액은 회사의 발행 주식 수와 현재 주가를 곱한 값으로, 회사의 전체 가치를 나타냅니다.\n예시: 애플의 시가총액이 2조 달러를 넘어서면서 S&P 500 지수에서 가장 큰 비중을 차지하게 되었습니다.\n연관키워드: 기업 가치, 주식, 투자\n\nDividend\n\n정의: 배당금은 기업이 주주들에게 이익의 일부를 현금으로 지급하는 것을 말합니다.\n예시: 코카콜라는 50년 이상 연속으로 배당금을 인상해온 S&P 500 기업 중 하나입니다.\n연관키워드: 주주 가치, 수익률, 투자 전략\n\nBlue Chip Stocks\n\n정의: 블루칩 주식은 재무적으로 안정적이고 오랜 기간 동안 꾸준한 실적을 보여온 대형 기업의 주식을 의미합니다.\n예시: 존슨앤존슨, 프록터앤갬블과 같은 기업들은 S&P 500에 포함된 대표적인 블루칩 주식입니다.\n연관키워드: 안정적 투자, 대형주, 배당주\n\nSector Rotation\n\n정의: 섹터 로테이션은 투자자들이 경제 사이클에 따라 다른 산업 섹터로 투자를 이동시키는 전략을 말합니다.\n예시: 경기 회복기에 투자자들이 기술주에서 금융주로 투자를 이동시키는 것이 섹터 로테이션의 한 예입니다.\n연관키워드: 투자 전략, 자산 배분, 경제 사이클\n\nEarnings Per Share (EPS)\n\n정의: 주당순이익(EPS)은 기업의 순이익을 발행 주식 수로 나눈 값으로, 주주들에게 돌아가는 이익을 나타냅니다.\n예시: 

In [16]:
# 결과 확인 
vector_store_persist.similarity_search("Blue Chip Stocks", k=1)

[Document(metadata={'source': './data/finance-keywords.txt'}, page_content='S&P 500\n\n정의: S&P 500은 미국 주식 시장에 상장된 500개의 대형 기업의 주가를 종합한 지수입니다. 이는 미국 경제와 주식 시장의 전반적인 상황을 나타내는 주요 지표로 사용됩니다.\n예시: 애플, 마이크로소프트, 아마존과 같은 대형 기술 기업들이 S&P 500에 포함되어 있습니다.\n연관키워드: 주식 시장, 지수, 대형주\n\nMarket Capitalization\n\n정의: 시가총액은 회사의 발행 주식 수와 현재 주가를 곱한 값으로, 회사의 전체 가치를 나타냅니다.\n예시: 애플의 시가총액이 2조 달러를 넘어서면서 S&P 500 지수에서 가장 큰 비중을 차지하게 되었습니다.\n연관키워드: 기업 가치, 주식, 투자\n\nDividend\n\n정의: 배당금은 기업이 주주들에게 이익의 일부를 현금으로 지급하는 것을 말합니다.\n예시: 코카콜라는 50년 이상 연속으로 배당금을 인상해온 S&P 500 기업 중 하나입니다.\n연관키워드: 주주 가치, 수익률, 투자 전략\n\nBlue Chip Stocks\n\n정의: 블루칩 주식은 재무적으로 안정적이고 오랜 기간 동안 꾸준한 실적을 보여온 대형 기업의 주식을 의미합니다.\n예시: 존슨앤존슨, 프록터앤갬블과 같은 기업들은 S&P 500에 포함된 대표적인 블루칩 주식입니다.\n연관키워드: 안정적 투자, 대형주, 배당주\n\nSector Rotation\n\n정의: 섹터 로테이션은 투자자들이 경제 사이클에 따라 다른 산업 섹터로 투자를 이동시키는 전략을 말합니다.\n예시: 경기 회복기에 투자자들이 기술주에서 금융주로 투자를 이동시키는 것이 섹터 로테이션의 한 예입니다.\n연관키워드: 투자 전략, 자산 배분, 경제 사이클\n\nEarnings Per Share (EPS)\n\n정의: 주당순이익(EPS)은 기업의 순이익을 발행 주식 수로 나눈 값으로, 주주들에게 돌아가는 이익

In [17]:
len(vector_store_persist.get())

7

# 문서 삭제 

`delete` 메서드는 벡터 저장소에서 지정된 ID에 해당하는 문서를 삭제하는 기능 제공

1. 주요 특징
    - 문서 ID 기반 삭제로 정확한 문서 관리가 가능
    - FAISS 인덱스와 문서 저장소 양쪽에서 삭제 수행
    - 삭제 후 인덱스 재정렬을 통해 데이터 일관성을 유지

2. 주의사항
    - 삭제 작업은 되돌릴 수 없으므로 신중하게 수행
    - 동시성 제어가 구현되어 있지 않아 다중 스레드 환경에서 주의 필요

In [18]:
# ID로 삭제 
delete_ids = ['text2_doc_2', 'text2_doc_3']
vector_store_persist.delete(delete_ids)

In [19]:
# 결과 재확인 
vector_store_persist.get('text2_doc_2')

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

## 검색기 변환

`as_retriever` 메서드는 현재 벡터 저장소를 기반으로 `VectorStoreRetriever` 객체를 생성하는 기능 제공

1. 매개변수
    - `**kwargs`: 검색 함수에 전달할 키워드 인자
       - `search_type` (Optional[str]): 검색 유형 (`similarity`, `mmr`, `similarity_score_threshold`)
       - `search_kwargs` (Optional[Dict]): 검색 함수에 전달할 추가 키워드 인자
            - `k`: 반환할 문서 수 (기본값: 4)
            - `score_threshold`: 최소 유사도 임계값
            - `fetch_k`: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
            - `lambda_mult`: MMR 결과의 다양성 조절 (0~1, 기본값: 0.5)
            - `filter`: 문서 메타데이터 필터링

In [20]:
# 검색기 변환 
retriever = vector_store_persist.as_retriever()
retriever.invoke('FAISS에 대해서 설명해 주세요')

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을

In [21]:
# MMR 검색 
retriever = vector_store_persist.as_retriever(
    search_type='mmr',
    search_kwargs={
        "k": 3, # 반환 문서수 (default : 4)
        "fetch_k": 5, # MMR알고리즘에 전달할 문서수 (default : 20)
        "lambda_mult": 0.25, # MMR결과 다양성 조절 (0~1, default: 0.5)
    }
)

retriever.invoke('FAISS에 대해서 설명해 주세요')

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을

In [22]:
# 상위 2개 문서만 검색되도록 
retriever = vector_store.as_retriever(
    search_type='mmr',
    search_kwargs={
        "k": 2, 
        "fetch_k": 5, 
    }
)

retriever.invoke('FAISS에 대해서 설명해 주세요')

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을

In [23]:
# 유사도 & 임계값 기반 검색되도록
retriever = vector_store.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={
        "score_threshold": 0.75, 
    }
)

retriever.invoke('FAISS에 대해서 설명해 주세요')

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을

In [24]:
# 가장 유사한 단일 문서만 검색되도록
retriever = vector_store.as_retriever(
    search_kwargs={
        "k": 1, 
    }
)

retriever.invoke('FAISS에 대해서 설명해 주세요')

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을

In [25]:
# 메타데이터 활용하여 필터가 필요할 때 
retriever = vector_store.as_retriever(
    search_kwargs={
        "filter": {'source':'./data/nlp-keywords.txt'}, 
        "k": 2,
    }
)

retriever.invoke('FAISS에 대해서 설명해 주세요')

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을

-----
** End of Documents **